## 1. Import Librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

# Configurazione per grafici più grandi e leggibili
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['font.size'] = 10

print("✅ Librerie importate con successo")

## 2. Caricamento Dati

In [ ]:
# Carica il file CSV
filename = 'accel_data_20251120_152940.csv'

df = pd.read_csv(filename)

# Mostra le prime righe
print(f"📊 Dataset caricato: {len(df)} campioni")
print(f"⏱️  Durata: {df['Timestamp'].max() - df['Timestamp'].min():.2f} secondi")
print(f"📈 Frequenza di campionamento: ~{len(df)/(df['Timestamp'].max() - df['Timestamp'].min()):.1f} Hz\n")

# Statistiche di base
print("Statistiche Magnitudine:")
print(df['Magnitude (g)'].describe())
print(f"\n📏 Range: {df['Magnitude (g)'].min():.4f}g - {df['Magnitude (g)'].max():.4f}g")

df.head(10)

## 3. Visualizzazione Dati Grezzi

In [ ]:
# Plot dei dati grezzi
fig, axes = plt.subplots(4, 1, figsize=(16, 10), sharex=True)

axes[0].plot(df['Timestamp'], df['X (g)'], 'r-', alpha=0.7, linewidth=1)
axes[0].set_ylabel('X (g)', fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_title('Bench Press - Accelerazione sui 3 assi + Magnitudine', fontsize=14, fontweight='bold')

axes[1].plot(df['Timestamp'], df['Y (g)'], 'g-', alpha=0.7, linewidth=1)
axes[1].set_ylabel('Y (g)', fontweight='bold')
axes[1].grid(True, alpha=0.3)

axes[2].plot(df['Timestamp'], df['Z (g)'], 'b-', alpha=0.7, linewidth=1)
axes[2].set_ylabel('Z (g)', fontweight='bold')
axes[2].grid(True, alpha=0.3)

axes[3].plot(df['Timestamp'], df['Magnitude (g)'], 'purple', linewidth=2, alpha=0.7)
axes[3].set_ylabel('Magnitudine (g)', fontweight='bold')
axes[3].set_xlabel('Tempo (s)', fontweight='bold')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🔍 Nota: Osserva il pattern di magnitudine - descrive chiaramente il movimento di discesa e salita del bilanciere")

## 4. Smoothing e Preprocessing

In [ ]:
# Applica smoothing gaussiano per ridurre il rumore
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

sigma = 2  # Parametro di smoothing

df['X_smooth'] = gaussian_filter1d(df['X (g)'], sigma=sigma)
df['Y_smooth'] = gaussian_filter1d(df['Y (g)'], sigma=sigma)
df['Z_smooth'] = gaussian_filter1d(df['Z (g)'], sigma=sigma)
df['Mag_smooth'] = gaussian_filter1d(df['Magnitude (g)'], sigma=sigma)

# Calcola la derivata (velocità) della magnitudine per identificare cambiamenti
df['Mag_velocity'] = np.gradient(df['Mag_smooth'], df['Timestamp'])
df['Mag_velocity_smooth'] = gaussian_filter1d(df['Mag_velocity'], sigma=sigma)

print("✅ Smoothing applicato con successo")
print(f"📊 Velocità magnitudine - Range: {df['Mag_velocity'].min():.2f} to {df['Mag_velocity'].max():.2f} g/s")

## 5. Identificazione Baseline (Lockout)

Il baseline rappresenta la posizione di lockout con le braccia estese.
Utilizziamo i primi 2 secondi di dati per stabilire il riferimento.

In [ ]:
# Identifica baseline dai primi 2 secondi
baseline_duration = 2.0
baseline_mask = df['Timestamp'] < (df['Timestamp'].min() + baseline_duration)

baseline_mag_mean = df.loc[baseline_mask, 'Magnitude (g)'].mean()
baseline_mag_std = df.loc[baseline_mask, 'Magnitude (g)'].std()
baseline_mag_min = baseline_mag_mean - 2 * baseline_mag_std
baseline_mag_max = baseline_mag_mean + 2 * baseline_mag_std

print("="*60)
print("BASELINE (LOCKOUT POSITION)")
print("="*60)
print(f"Mean:     {baseline_mag_mean:.4f}g")
print(f"Std Dev:  {baseline_mag_std:.4f}g")
print(f"Range:    {baseline_mag_min:.4f}g - {baseline_mag_max:.4f}g")
print(f"Samples:  {baseline_mask.sum()}")
print("\n🔒 Questa è la magnitudine quando il bilanciere è in lockout (braccia estese)")

## 6. Rilevamento Rep Bottoms (Chest Touch)

Identifichiamo i punti di minima magnitudine che corrispondono al tocco del bilanciere sul petto.

In [ ]:
# Trova i minimi locali (chest touch points)
# Inverti il segnale per trovare i minimi come picchi
inverted_mag = -df['Mag_smooth'].values

# Parametri ottimizzati per bench press
prominence = 0.05  # Differenza minima per essere considerato un rep
min_distance = 50  # Distanza minima tra reps (~1 secondo a 50Hz)

bottoms_idx, bottom_properties = find_peaks(inverted_mag, 
                                             prominence=prominence, 
                                             distance=min_distance)

print("="*60)
print(f"REP BOTTOMS DETECTED: {len(bottoms_idx)}")
print("="*60)

# Crea DataFrame con dettagli dei bottoms
bottoms_df = pd.DataFrame({
    'Rep': range(1, len(bottoms_idx) + 1),
    'Time (s)': df.loc[bottoms_idx, 'Timestamp'].values,
    'Magnitude (g)': df.loc[bottoms_idx, 'Magnitude (g)'].values,
    'Depth from Baseline': baseline_mag_mean - df.loc[bottoms_idx, 'Magnitude (g)'].values
})

print(bottoms_df.to_string(index=False))
print(f"\n📊 Profondità media: {bottoms_df['Depth from Baseline'].mean():.4f}g")
print(f"📊 Magnitudine media al bottom: {bottoms_df['Magnitude (g)'].mean():.4f}g")

## 7. Rilevamento Picchi Concentrici

I picchi di magnitudine durante la fase concentrica indicano l'accelerazione esplosiva del bilanciere.

In [ ]:
# Trova i picchi (concentric power)
peaks_idx, peak_properties = find_peaks(df['Mag_smooth'].values, 
                                        prominence=prominence, 
                                        distance=min_distance)

print("="*60)
print(f"CONCENTRIC PEAKS DETECTED: {len(peaks_idx)}")
print("="*60)

# Crea DataFrame con dettagli dei picchi
peaks_df = pd.DataFrame({
    'Peak': range(1, len(peaks_idx) + 1),
    'Time (s)': df.loc[peaks_idx, 'Timestamp'].values,
    'Magnitude (g)': df.loc[peaks_idx, 'Magnitude (g)'].values,
    'Power above Baseline': df.loc[peaks_idx, 'Magnitude (g)'].values - baseline_mag_mean
})

print(peaks_df.to_string(index=False))
print(f"\n💪 Potenza media: {peaks_df['Power above Baseline'].mean():.4f}g sopra baseline")
print(f"💪 Picco massimo: {peaks_df['Magnitude (g)'].max():.4f}g")

## 8. Calcolo Durate Rep

Analizza la durata di ogni ripetizione (da bottom a bottom successivo).

In [ ]:
if len(bottoms_idx) > 1:
    # Calcola durate tra bottoms consecutivi
    rep_times = df.loc[bottoms_idx, 'Timestamp'].values
    rep_durations = np.diff(rep_times)
    
    print("="*60)
    print("REP TIMING ANALYSIS")
    print("="*60)
    print(f"Numero di reps: {len(bottoms_idx)}")
    print(f"Numero di intervalli: {len(rep_durations)}")
    print(f"\nDurata media rep: {rep_durations.mean():.2f}s")
    print(f"Durata minima rep: {rep_durations.min():.2f}s")
    print(f"Durata massima rep: {rep_durations.max():.2f}s")
    print(f"Deviazione standard: {rep_durations.std():.2f}s")
    
    print("\n📊 Durate individuali (secondi):")
    for i, duration in enumerate(rep_durations):
        print(f"  Rep {i+1} → Rep {i+2}: {duration:.2f}s")
else:
    print("⚠️  Non abbastanza reps per calcolare le durate")

## 9. Rilevamento Fasi Complete

Identifichiamo tutte le fasi di ogni ripetizione:
1. **Baseline** - Posizione di lockout
2. **Eccentric Start** - Inizio discesa (velocità negativa)
3. **Bottom** - Chest touch (magnitudine minima)
4. **Concentric Start** - Inizio spinta (velocità positiva)
5. **Peak** - Picco di potenza
6. **Return to Lockout** - Ritorno a baseline

In [ ]:
# Definisci soglie per il rilevamento delle fasi
VELOCITY_THRESHOLD = 0.5  # g/s per rilevare inizio movimento

# Inizializza lista per memorizzare le fasi
phases = []
current_phase = 'BASELINE'
phase_start_idx = 0

# Soglie per identificare le fasi
BASELINE_LOWER = baseline_mag_mean - baseline_mag_std
BASELINE_UPPER = baseline_mag_mean + baseline_mag_std

for i in range(len(df)):
    mag = df.loc[i, 'Mag_smooth']
    vel = df.loc[i, 'Mag_velocity_smooth']
    
    if current_phase == 'BASELINE':
        # Cerca inizio discesa (velocità negativa significativa)
        if vel < -VELOCITY_THRESHOLD and mag < BASELINE_LOWER:
            phases.append({
                'phase': current_phase,
                'start_idx': phase_start_idx,
                'end_idx': i,
                'duration': df.loc[i, 'Timestamp'] - df.loc[phase_start_idx, 'Timestamp']
            })
            current_phase = 'ECCENTRIC'
            phase_start_idx = i
    
    elif current_phase == 'ECCENTRIC':
        # Cerca bottom (magnitudine molto bassa + velocità vicina a zero)
        if abs(vel) < VELOCITY_THRESHOLD and mag < 0.7:
            phases.append({
                'phase': current_phase,
                'start_idx': phase_start_idx,
                'end_idx': i,
                'duration': df.loc[i, 'Timestamp'] - df.loc[phase_start_idx, 'Timestamp']
            })
            current_phase = 'BOTTOM'
            phase_start_idx = i
    
    elif current_phase == 'BOTTOM':
        # Cerca inizio concentrica (velocità positiva)
        if vel > VELOCITY_THRESHOLD:
            phases.append({
                'phase': current_phase,
                'start_idx': phase_start_idx,
                'end_idx': i,
                'duration': df.loc[i, 'Timestamp'] - df.loc[phase_start_idx, 'Timestamp']
            })
            current_phase = 'CONCENTRIC'
            phase_start_idx = i
    
    elif current_phase == 'CONCENTRIC':
        # Cerca ritorno a baseline (magnitudine torna al baseline range)
        if mag > BASELINE_LOWER and mag < BASELINE_UPPER and abs(vel) < VELOCITY_THRESHOLD:
            phases.append({
                'phase': current_phase,
                'start_idx': phase_start_idx,
                'end_idx': i,
                'duration': df.loc[i, 'Timestamp'] - df.loc[phase_start_idx, 'Timestamp']
            })
            current_phase = 'BASELINE'
            phase_start_idx = i

# Aggiungi l'ultima fase
phases.append({
    'phase': current_phase,
    'start_idx': phase_start_idx,
    'end_idx': len(df) - 1,
    'duration': df.loc[len(df)-1, 'Timestamp'] - df.loc[phase_start_idx, 'Timestamp']
})

# Crea DataFrame delle fasi
phases_df = pd.DataFrame(phases)

print("="*60)
print(f"FASI RILEVATE: {len(phases_df)}")
print("="*60)
print(phases_df[['phase', 'duration']].to_string(index=False))

# Statistiche per fase
print("\n📊 Statistiche per Fase:")
phase_stats = phases_df.groupby('phase')['duration'].agg(['count', 'mean', 'std', 'min', 'max'])
print(phase_stats)

## 9b. Analisi Rebound Effect (Rinculo Post-Lockout)

⚠️ **FENOMENO CRITICO**: Dopo il lockout al top, l'accelerometro registra DUE dip distinti!

**SEQUENZA CORRETTA:**
```
1. Concentric Peak 🟢 (lockout al top - alta magnitudine)
      ↓
2. Ritorno verso baseline (discesa rapida)
      ↓
3. Rebound Spike ⚠️ (PRIMO dip - FALSO!)
   └─ Artefatto da frenata brusca al lockout
      ↓
4. Piccola risalita (stabilizzazione ~0.1s)
      ↓
5. Chest Touch 🎯 (SECONDO dip - VERO tocco sul petto!)
   └─ Piccolo "su-giù" che rappresenta il tocco reale
      ↓
6. Inizio nuova concentrica
```

**IMPORTANTE:** Il rebound è il PRIMO dip (falso), il chest touch è il SECONDO dip (vero)!

In [ ]:
"""
""" REBOUND DETECTION ALGORITHM CORRETTO
Identifica il rinculo post-lockout separandolo dal vero chest touch

SEQUENZA CORRETTA:
1. Concentric Peak (lockout al top)
2. Ritorno verso baseline
3. REBOUND SPIKE (primo dip - FALSO, artefatto da rinculo)
4. Piccola risalita (stabilizzazione)
5. CHEST TOUCH (secondo dip più piccolo - VERO tocco sul petto)
"""

print("="*90)
print("🔍 ANALISI REBOUND EFFECT - Identificazione Rinculo vs Chest Touch")
print("="*90)

# Parametri per identificare rebound e true bottom
REBOUND_SEARCH_WINDOW = 1.0  # secondi - cerca il primo minimum (rebound) in questa finestra
TRUE_BOTTOM_SEARCH_START = 0.3  # secondi - inizia a cercare true bottom DOPO il rebound
MIN_REBOUND_DROP = 0.10  # g - drop minimo per identificare rebound

# Liste per memorizzare risultati
rebound_events = []
chest_touches = []

# Per ogni picco concentrico, cerca prima il rebound, poi il true bottom
for peak_idx in peaks_idx:
    peak_time = df.loc[peak_idx, 'Timestamp']
    peak_mag = df.loc[peak_idx, 'Magnitude (g)']
    
    # Trova il prossimo picco per limitare la ricerca
    next_peak_idx = peaks_idx[peaks_idx > peak_idx].min() if any(peaks_idx > peak_idx) else len(df)
    next_peak_time = df.loc[next_peak_idx, 'Timestamp'] if next_peak_idx < len(df) else df['Timestamp'].max()
    
    # STEP 1: Cerca il REBOUND (primo minimum dopo il picco)
    rebound_search_end = min(peak_time + REBOUND_SEARCH_WINDOW, next_peak_time)
    rebound_mask = (df['Timestamp'] > peak_time) & (df['Timestamp'] <= rebound_search_end)
    
    rebound_idx = None
    if rebound_mask.any():
        rebound_candidates = df[rebound_mask]
        rebound_idx = rebound_candidates['Mag_smooth'].idxmin()
        rebound_mag = df.loc[rebound_idx, 'Mag_smooth']
        rebound_time = df.loc[rebound_idx, 'Timestamp']
        rebound_drop = peak_mag - rebound_mag
        
        # Verifica se c'è un drop significativo (è davvero un rebound)
        if rebound_drop > MIN_REBOUND_DROP:
            rebound_events.append({
                'peak_idx': peak_idx,
                'rebound_idx': rebound_idx,
                'peak_mag': peak_mag,
                'rebound_mag': rebound_mag,
                'rebound_drop': rebound_drop,
                'time_after_peak': rebound_time - peak_time
            })
    
    # STEP 2: Cerca il CHEST TOUCH dopo il rebound
    # Se abbiamo trovato un rebound, cerca DOPO di esso
    # Altrimenti cerca dopo la finestra standard
    if rebound_idx is not None:
        chest_touch_search_start_time = df.loc[rebound_idx, 'Timestamp'] + 0.1  # 100ms dopo rebound
    else:
        chest_touch_search_start_time = peak_time + TRUE_BOTTOM_SEARCH_START
    
    # Finestra per cercare chest touch (dopo rebound, prima del prossimo picco)
    chest_touch_mask = (df['Timestamp'] > chest_touch_search_start_time) & (df['Timestamp'] < next_peak_time)
    
    if chest_touch_mask.any():
        chest_touch_candidates = df[chest_touch_mask]
        chest_touch_idx = chest_touch_candidates['Mag_smooth'].idxmin()
        chest_touch_mag = df.loc[chest_touch_idx, 'Mag_smooth']
        chest_touch_time = df.loc[chest_touch_idx, 'Timestamp']
        
        chest_touches.append({
            'peak_idx': peak_idx,
            'rebound_idx': rebound_idx if rebound_idx is not None else None,
            'chest_touch_idx': chest_touch_idx,
            'chest_touch_mag': chest_touch_mag,
            'chest_touch_time': chest_touch_time,
            'time_after_peak': chest_touch_time - peak_time,
            'time_after_rebound': (chest_touch_time - df.loc[rebound_idx, 'Timestamp']) if rebound_idx is not None else None,
            'depth': baseline_mag_mean - chest_touch_mag
        })

# Converti in DataFrame
rebound_df = pd.DataFrame(rebound_events)
chest_touch_df = pd.DataFrame(chest_touches)

print(f"\n💥 REBOUND EVENTS DETECTED: {len(rebound_df)}")
if len(rebound_df) > 0:
    print(f"\n{'Rebound #':<12} {'Time After Peak':<18} {'Peak→Rebound Drop':<22} {'Rebound Mag':<15}")
    print("-" * 67)
    for i, row in rebound_df.iterrows():
        print(f"  {i+1:<10} {row['time_after_peak']:.3f}s{'':<12} {row['rebound_drop']:.4f}g{'':<14} {row['rebound_mag']:.4f}g")
    
    print(f"\n📊 Statistiche Rebound:")
    print(f"  • Tempo medio dopo picco: {rebound_df['time_after_peak'].mean():.3f}s (±{rebound_df['time_after_peak'].std():.3f}s)")
    print(f"  • Drop medio magnitudine: {rebound_df['rebound_drop'].mean():.4f}g (±{rebound_df['rebound_drop'].std():.4f}g)")
    print(f"  • Magnitudine media rebound: {rebound_df['rebound_mag'].mean():.4f}g")

print(f"\n🎯 CHEST TOUCHES DETECTED: {len(chest_touch_df)}")
if len(chest_touch_df) > 0:
    print(f"\n{'Touch #':<12} {'Time After Peak':<18} {'Touch Mag':<18} {'Depth':<15}")
    print("-" * 63)
    for i, row in chest_touch_df.iterrows():
        print(f"  {i+1:<10} {row['time_after_peak']:.3f}s{'':<12} {row['chest_touch_mag']:.4f}g{'':<10} {row['depth']:.4f}g")
    
    print(f"\n📊 Statistiche Chest Touches:")
    print(f"  • Tempo medio dopo picco: {chest_touch_df['time_after_peak'].mean():.3f}s (±{chest_touch_df['time_after_peak'].std():.3f}s)")
    print(f"  • Magnitudine media: {chest_touch_df['chest_touch_mag'].mean():.4f}g (±{chest_touch_df['chest_touch_mag'].std():.4f}g)")
    print(f"  • Profondità media: {chest_touch_df['depth'].mean():.4f}g (±{chest_touch_df['depth'].std():.4f}g)")
    
    # Mostra tempo dopo rebound se disponibile
    if 'time_after_rebound' in chest_touch_df.columns:
        valid_times = chest_touch_df['time_after_rebound'].dropna()
        if len(valid_times) > 0:
            print(f"  • Tempo medio dopo rebound: {valid_times.mean():.3f}s (±{valid_times.std():.3f}s)")

print("\n" + "="*90)

# Confronto: Rebound vs Chest Touch
if len(rebound_df) > 0 and len(chest_touch_df) > 0:
    print("\n⚖️  CONFRONTO REBOUND (PRIMO DIP) vs CHEST TOUCH (SECONDO DIP):")
    print("-" * 90)
    print(f"  {'Metrica':<35} {'Rebound (1° dip - Falso)':<30} {'Chest Touch (2° dip - Vero)':<25}")
    print("-" * 90)
    print(f"  {'Tempo dopo picco':<35} {rebound_df['time_after_peak'].mean():.3f}s{'':<23} {chest_touch_df['time_after_peak'].mean():.3f}s")
    print(f"  {'Magnitudine':<35} {rebound_df['rebound_mag'].mean():.4f}g{'':<21} {chest_touch_df['chest_touch_mag'].mean():.4f}g")
    
    # Differenza temporale
    time_diff = chest_touch_df['time_after_peak'].mean() - rebound_df['time_after_peak'].mean()
    print(f"\n  ⏱️  SEQUENZA: Peak → Rebound (dopo ~{rebound_df['time_after_peak'].mean():.2f}s) → Chest Touch (dopo altri ~{time_diff:.2f}s)")
    print(f"  📉 Differenza magnitudine: {abs(rebound_df['rebound_mag'].mean() - chest_touch_df['chest_touch_mag'].mean()):.4f}g")
    
    print("\n✅ CONCLUSIONE:")
    print("  • REBOUND = PRIMO dip sotto baseline (artefatto da frenata brusca)")
    print("  • CHEST TOUCH = SECONDO dip (vero tocco sul petto, piccolo su-giù)")
    print("  • Sequenza: Peak → Baseline → Rebound (falso) → Stabilizzazione → Chest Touch (vero)")
    print("="*90)

## 9c. Visualizzazione Rebound vs True Bottom

Grafico dedicato per confrontare visivamente il rebound spike e il vero chest touch.

In [ ]:
if len(rebound_df) > 0 and len(chest_touch_df) > 0:
    # Crea plot dettagliato per prime 2-3 reps
    num_reps_to_show = min(3, len(rebound_df))
    
    fig, axes = plt.subplots(num_reps_to_show, 1, figsize=(16, 5 * num_reps_to_show))
    if num_reps_to_show == 1:
        axes = [axes]
    
    for i in range(num_reps_to_show):
        ax = axes[i]
        
        # Trova indici per questa rep
        peak_idx = rebound_df.loc[i, 'peak_idx']
        rebound_idx = rebound_df.loc[i, 'rebound_idx']
        chest_touch_idx = chest_touch_df.loc[i, 'chest_touch_idx']
        
        # Trova finestra temporale da mostrare (±1 secondo dal picco)
        peak_time = df.loc[peak_idx, 'Timestamp']
        window_start = peak_time - 0.5
        window_end = peak_time + 2.0
        window_mask = (df['Timestamp'] >= window_start) & (df['Timestamp'] <= window_end)
        
        # Plot magnitudine
        ax.plot(df.loc[window_mask, 'Timestamp'], 
                df.loc[window_mask, 'Magnitude (g)'], 
                'lightgray', alpha=0.5, linewidth=1, label='Raw')
        ax.plot(df.loc[window_mask, 'Timestamp'], 
                df.loc[window_mask, 'Mag_smooth'], 
                'blue', linewidth=2.5, label='Smoothed')
        
        # Baseline
        ax.axhline(y=baseline_mag_mean, color='green', linestyle='--', 
                   linewidth=2, alpha=0.5, label='Baseline (Lockout)')
        
        # Markers
        # 1. Concentric Peak
        ax.scatter(df.loc[peak_idx, 'Timestamp'], 
                   df.loc[peak_idx, 'Magnitude (g)'],
                   s=250, marker='^', color='lime', edgecolors='darkgreen', 
                   linewidths=3, zorder=10, label='Concentric Peak')
        
        # 2. Rebound Spike (FALSO bottom)
        ax.scatter(df.loc[rebound_idx, 'Timestamp'], 
                   df.loc[rebound_idx, 'Mag_smooth'],
                   s=250, marker='X', color='orange', edgecolors='darkorange', 
                   linewidths=3, zorder=10, label='⚠️  Rebound Spike (FALSO)')
        
        # 3. Chest Touch (vero tocco sul petto)
        ax.scatter(df.loc[chest_touch_idx, 'Timestamp'], 
                   df.loc[chest_touch_idx, 'Mag_smooth'],
                   s=250, marker='v', color='red', edgecolors='darkred', 
                   linewidths=3, zorder=10, label='🎯 Chest Touch (VERO)')
        
        # Zona rebound (area grigia)
        rebound_window_end = peak_time + REBOUND_SKIP_TIME
        ax.axvspan(peak_time, rebound_window_end, 
                   color='yellow', alpha=0.2, zorder=1, label=f'Rebound Window ({REBOUND_SKIP_TIME}s)')
        
        # Annotazioni con frecce
        # Freccia Peak → Rebound
        ax.annotate('', xy=(df.loc[rebound_idx, 'Timestamp'], df.loc[rebound_idx, 'Mag_smooth']),
                    xytext=(df.loc[peak_idx, 'Timestamp'], df.loc[peak_idx, 'Magnitude (g)']),
                    arrowprops=dict(arrowstyle='->', color='orange', lw=2, linestyle='--'))
        
        # Freccia Rebound → Chest Touch
        ax.annotate('', xy=(df.loc[chest_touch_idx, 'Timestamp'], df.loc[chest_touch_idx, 'Mag_smooth']),
                    xytext=(df.loc[rebound_idx, 'Timestamp'], df.loc[rebound_idx, 'Mag_smooth']),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2, linestyle='--'))
        
        # Testo informativo
        rebound_time = df.loc[rebound_idx, 'Timestamp'] - peak_time
        chest_touch_time = df.loc[chest_touch_idx, 'Timestamp'] - peak_time
        time_diff = chest_touch_time - rebound_time
        
        info_text = (f"Rep {i+1} - SEQUENZA:\n"
                    f"1. Peak (lockout)\n"
                    f"2. Rebound (1° dip FALSO): +{rebound_time:.3f}s\n"
                    f"3. Chest Touch (2° dip VERO): +{chest_touch_time:.3f}s\n"
                    f"Differenza: {time_diff:.3f}s")
        
        ax.text(0.02, 0.98, info_text, transform=ax.transAxes,
                fontsize=9, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
        
        ax.set_ylabel('Magnitudine (g)', fontweight='bold', fontsize=11)
        ax.set_xlabel('Tempo (s)', fontweight='bold', fontsize=11)
        ax.set_title(f'Rep {i+1}: Peak → Rebound (1° dip FALSO) → Chest Touch (2° dip VERO)', 
                     fontsize=13, fontweight='bold')
        ax.legend(loc='upper right', fontsize=9)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizzazione rebound effect completata!")
    print("💡 NOTA IMPORTANTE:")
    print("  • PRIMO dip (X arancione) = REBOUND SPIKE (falso, artefatto)")
    print("  • SECONDO dip (▼ rosso) = CHEST TOUCH (vero tocco sul petto)")
    print("  • Sequenza: Peak verde → Baseline → Rebound (1°) → Chest Touch (2°)")
else:
    print("⚠️  Dati insufficienti per visualizzare rebound effect")

## 10. Visualizzazione Completa con Fasi Annotate

In [ ]:
# Crea una visualizzazione completa con tutte le annotazioni
fig = plt.figure(figsize=(18, 12))

# Layout: 3 righe
gs = fig.add_gridspec(3, 1, height_ratios=[2, 1, 1], hspace=0.3)

# Plot 1: Magnitudine con fasi e markers
ax1 = fig.add_subplot(gs[0])

# Plot magnitudine
ax1.plot(df['Timestamp'], df['Magnitude (g)'], 'b-', alpha=0.3, linewidth=1, label='Raw')
ax1.plot(df['Timestamp'], df['Mag_smooth'], 'b-', linewidth=2, label='Smoothed')

# Baseline range
ax1.axhline(y=baseline_mag_mean, color='green', linestyle='--', linewidth=2, 
            label=f'Baseline: {baseline_mag_mean:.3f}g', alpha=0.7)
ax1.fill_between(df['Timestamp'], baseline_mag_min, baseline_mag_max, 
                 color='green', alpha=0.1, label='Baseline Range')

# Marca i bottoms (chest touch)
if len(bottoms_idx) > 0:
    ax1.scatter(df.loc[bottoms_idx, 'Timestamp'], 
                df.loc[bottoms_idx, 'Magnitude (g)'], 
                color='red', s=150, zorder=5, marker='v',
                label=f'Chest Touch ({len(bottoms_idx)} reps)', 
                edgecolors='darkred', linewidths=2)

# Marca i picchi concentrici
if len(peaks_idx) > 0:
    ax1.scatter(df.loc[peaks_idx, 'Timestamp'], 
                df.loc[peaks_idx, 'Magnitude (g)'], 
                color='lime', s=150, zorder=5, marker='^',
                label=f'Concentric Peaks ({len(peaks_idx)})', 
                edgecolors='darkgreen', linewidths=2)

# Marca i REBOUND SPIKES (artefatti falsi bottoms)
if len(rebound_df) > 0:
    rebound_indices = rebound_df['rebound_idx'].values
    ax1.scatter(df.loc[rebound_indices, 'Timestamp'], 
                df.loc[rebound_indices, 'Mag_smooth'], 
                color='orange', s=120, zorder=6, marker='X',
                label=f'⚠️  Rebound Spikes ({len(rebound_df)})', 
                edgecolors='darkorange', linewidths=2)

# Marca i CHEST TOUCHES (veri tocchi sul petto)
if len(chest_touch_df) > 0:
    chest_touch_indices = chest_touch_df['chest_touch_idx'].values
    ax1.scatter(df.loc[chest_touch_indices, 'Timestamp'], 
                df.loc[chest_touch_indices, 'Mag_smooth'], 
                color='darkred', s=150, zorder=7, marker='v',
                label=f'🎯 Chest Touches ({len(chest_touch_df)})', 
                edgecolors='black', linewidths=2)

# Colora le fasi
phase_colors = {
    'BASELINE': 'lightgreen',
    'ECCENTRIC': 'lightcoral',
    'BOTTOM': 'yellow',
    'CONCENTRIC': 'lightblue'
}

for _, phase in phases_df.iterrows():
    start_time = df.loc[phase['start_idx'], 'Timestamp']
    end_time = df.loc[phase['end_idx'], 'Timestamp']
    ax1.axvspan(start_time, end_time, alpha=0.2, 
                color=phase_colors.get(phase['phase'], 'gray'))

ax1.set_ylabel('Magnitudine (g)', fontweight='bold', fontsize=12)
ax1.set_title('Bench Press - Analisi Completa con Fasi', fontsize=16, fontweight='bold')
ax1.legend(loc='upper right', fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: Velocità della magnitudine
ax2 = fig.add_subplot(gs[1], sharex=ax1)
ax2.plot(df['Timestamp'], df['Mag_velocity_smooth'], 'purple', linewidth=2)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5, linewidth=1)
ax2.axhline(y=VELOCITY_THRESHOLD, color='blue', linestyle=':', alpha=0.5, 
            label=f'Threshold: ±{VELOCITY_THRESHOLD}')
ax2.axhline(y=-VELOCITY_THRESHOLD, color='blue', linestyle=':', alpha=0.5)
ax2.fill_between(df['Timestamp'], -VELOCITY_THRESHOLD, VELOCITY_THRESHOLD, 
                 color='gray', alpha=0.1)
ax2.set_ylabel('Velocità (g/s)', fontweight='bold', fontsize=12)
ax2.set_title('Rate of Change - Eccentric (negativo) vs Concentric (positivo)', fontsize=12)
ax2.legend(loc='upper right', fontsize=9)
ax2.grid(True, alpha=0.3)

# Plot 3: Assi individuali
ax3 = fig.add_subplot(gs[2], sharex=ax1)
ax3.plot(df['Timestamp'], df['X_smooth'], 'r-', alpha=0.7, linewidth=1.5, label='X axis')
ax3.plot(df['Timestamp'], df['Y_smooth'], 'g-', alpha=0.7, linewidth=1.5, label='Y axis')
ax3.plot(df['Timestamp'], df['Z_smooth'], 'b-', alpha=0.7, linewidth=1.5, label='Z axis')
ax3.set_xlabel('Tempo (s)', fontweight='bold', fontsize=12)
ax3.set_ylabel('Accelerazione (g)', fontweight='bold', fontsize=12)
ax3.set_title('Assi Individuali (Smoothed)', fontsize=12)
ax3.legend(loc='upper right', fontsize=9)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Visualizzazione completa generata!")

## 11. Summary Report

Riepilogo completo dell'analisi del bench press.

In [ ]:
print("="*70)
print(" "*20 + "BENCH PRESS ANALYSIS REPORT")
print("="*70)

print("\n📊 DATASET INFO")
print(f"  • File: {filename}")
print(f"  • Duration: {df['Timestamp'].max() - df['Timestamp'].min():.2f} seconds")
print(f"  • Samples: {len(df)}")
print(f"  • Sampling Rate: ~{len(df)/(df['Timestamp'].max() - df['Timestamp'].min()):.1f} Hz")

print("\n🔒 BASELINE (LOCKOUT)")
print(f"  • Mean Magnitude: {baseline_mag_mean:.4f}g")
print(f"  • Std Deviation: {baseline_mag_std:.4f}g")
print(f"  • Range: {baseline_mag_min:.4f}g - {baseline_mag_max:.4f}g")

print(f"\n🏋️ REPS DETECTED: {len(bottoms_idx)}")
if len(bottoms_idx) > 0:
    print(f"  • Average Bottom Magnitude: {bottoms_df['Magnitude (g)'].mean():.4f}g")
    print(f"  • Average Depth: {bottoms_df['Depth from Baseline'].mean():.4f}g")
    print(f"  • Min Bottom: {bottoms_df['Magnitude (g)'].min():.4f}g")
    print(f"  • Max Bottom: {bottoms_df['Magnitude (g)'].max():.4f}g")

print(f"\n💪 CONCENTRIC POWER")
if len(peaks_idx) > 0:
    print(f"  • Number of Peaks: {len(peaks_idx)}")
    print(f"  • Average Peak: {peaks_df['Magnitude (g)'].mean():.4f}g")
    print(f"  • Max Peak: {peaks_df['Magnitude (g)'].max():.4f}g")
    print(f"  • Average Power above Baseline: {peaks_df['Power above Baseline'].mean():.4f}g")

print("\n⏱️  TIMING")
if len(rep_durations) > 0:
    print(f"  • Average Rep Duration: {rep_durations.mean():.2f}s")
    print(f"  • Rep Duration Range: {rep_durations.min():.2f}s - {rep_durations.max():.2f}s")
    print(f"  • Tempo Consistency (Std): {rep_durations.std():.2f}s")

print("\n🎯 PHASE BREAKDOWN")
for phase_name in ['BASELINE', 'ECCENTRIC', 'BOTTOM', 'CONCENTRIC']:
    phase_data = phases_df[phases_df['phase'] == phase_name]
    if len(phase_data) > 0:
        print(f"  • {phase_name:12s}: {len(phase_data):2d} occurrences, "
              f"avg {phase_data['duration'].mean():.2f}s ± {phase_data['duration'].std():.2f}s")

print("\n📈 MAGNITUDE STATISTICS")
print(f"  • Overall Range: {df['Magnitude (g)'].min():.4f}g - {df['Magnitude (g)'].max():.4f}g")
print(f"  • Mean: {df['Magnitude (g)'].mean():.4f}g")
print(f"  • Std: {df['Magnitude (g)'].std():.4f}g")
print(f"  • Total Excursion: {df['Magnitude (g)'].max() - df['Magnitude (g)'].min():.4f}g")

print("\n🔍 COMPARISON TO OTHER LIFTS")
print("  • Pattern: Eccentric → Bottom → Concentric (similar to Squat)")
print("  • Impact Events: NO (unlike Deadlift)")
print("  • ROM: Shorter than Squat")
print("  • Baseline Stability: High (lockout position)")
print(f"  • Depth Drop: {bottoms_df['Depth from Baseline'].mean():.4f}g "
      f"({(bottoms_df['Depth from Baseline'].mean()/baseline_mag_mean)*100:.1f}% of baseline)")

print("\n" + "="*70)
print("✅ Analysis Complete!")
print("="*70)

## 12. Recommended Thresholds for Production

Questi valori possono essere utilizzati per configurare un sistema di rilevamento automatico.

In [ ]:
print("="*70)
print("RECOMMENDED CONFIGURATION FOR BENCH PRESS DETECTION")
print("="*70)

print("\n# Bench Press Configuration")
print("BENCH_PRESS_CONFIG = {")
print(f"    # Baseline (Lockout)")
print(f"    'baseline_mag_mean': {baseline_mag_mean:.4f},")
print(f"    'baseline_mag_std': {baseline_mag_std:.4f},")
print(f"    'baseline_mag_min': {baseline_mag_min:.4f},")
print(f"    'baseline_mag_max': {baseline_mag_max:.4f},")
print(f"    ")
print(f"    # Thresholds")
print(f"    'bottom_threshold': {bottoms_df['Magnitude (g)'].max():.4f},  # Max magnitude at bottom")
print(f"    'eccentric_velocity_threshold': -{VELOCITY_THRESHOLD},  # Negative for descent")
print(f"    'concentric_velocity_threshold': {VELOCITY_THRESHOLD},  # Positive for press")
print(f"    ")
print(f"    # Peak Detection")
print(f"    'prominence': {prominence},")
print(f"    'min_distance': {min_distance},  # samples (~{min_distance/50:.1f}s at 50Hz)")
print(f"    ")
print(f"    # Timing")
print(f"    'min_rep_duration': {rep_durations.min():.2f},  # seconds")
print(f"    'max_rep_duration': {rep_durations.max():.2f},  # seconds")
print(f"    'expected_rep_duration': {rep_durations.mean():.2f},  # seconds")
print(f"    ")
print(f"    # Smoothing")
print(f"    'gaussian_sigma': {sigma},")
print("}")

print("\n" + "="*70)
print("\n💡 USAGE NOTES:")
print("  • Use baseline_mag_min/max to detect lockout position")
print("  • Use bottom_threshold to identify chest touch")
print("  • Use velocity thresholds to detect phase transitions")
print("  • Prominence and min_distance for peak detection (scipy.signal.find_peaks)")
print("  • Rep duration for validation and outlier detection")

## 13. Confronto Rep-by-Rep

Analisi dettagliata di ogni singola ripetizione.

In [ ]:
if len(bottoms_idx) > 1:
    print("="*80)
    print("REP-BY-REP ANALYSIS")
    print("="*80)
    
    for i in range(len(bottoms_idx) - 1):
        rep_num = i + 1
        start_idx = bottoms_idx[i]
        end_idx = bottoms_idx[i + 1]
        
        # Estrai dati per questo rep
        rep_data = df.iloc[start_idx:end_idx]
        
        # Calcola metriche
        rep_duration = df.loc[end_idx, 'Timestamp'] - df.loc[start_idx, 'Timestamp']
        bottom_mag = df.loc[start_idx, 'Magnitude (g)']
        max_mag = rep_data['Magnitude (g)'].max()
        depth = baseline_mag_mean - bottom_mag
        power = max_mag - baseline_mag_mean
        
        # Trova il tempo del picco
        peak_in_rep = rep_data['Magnitude (g)'].idxmax()
        time_to_peak = df.loc[peak_in_rep, 'Timestamp'] - df.loc[start_idx, 'Timestamp']
        
        print(f"\n🏋️  REP {rep_num}")
        print(f"  Time: {df.loc[start_idx, 'Timestamp']:.2f}s - {df.loc[end_idx, 'Timestamp']:.2f}s")
        print(f"  Duration: {rep_duration:.2f}s")
        print(f"  Bottom Magnitude: {bottom_mag:.4f}g")
        print(f"  Depth: {depth:.4f}g ({(depth/baseline_mag_mean)*100:.1f}% of baseline)")
        print(f"  Peak Magnitude: {max_mag:.4f}g")
        print(f"  Power: {power:.4f}g above baseline")
        print(f"  Time to Peak: {time_to_peak:.2f}s ({(time_to_peak/rep_duration)*100:.1f}% of rep)")
    
    print("\n" + "="*80)

## 14. Key Insights and Patterns

Osservazioni chiave emerse dall'analisi.

In [ ]:
print("="*80)
print("KEY INSIGHTS FROM BENCH PRESS DATA")
print("="*80)

print("\n1️⃣  BASELINE CHARACTERISTICS")
print(f"   • Very stable lockout position: {baseline_mag_std:.4f}g std deviation")
print(f"   • Mean magnitude at lockout: ~{baseline_mag_mean:.3f}g")
print("   • This provides excellent reference for detecting rep start/end")

print("\n2️⃣  ECCENTRIC PHASE (Bar Descent)")
avg_depth = bottoms_df['Depth from Baseline'].mean()
print(f"   • Average magnitude drop: {avg_depth:.4f}g")
print(f"   • Represents {(avg_depth/baseline_mag_mean)*100:.1f}% decrease from baseline")
print("   • Pattern is consistent and predictable")

print("\n3️⃣  BOTTOM PHASE (Chest Touch)")
print(f"   • Minimum magnitudes range: {bottoms_df['Magnitude (g)'].min():.4f}g - {bottoms_df['Magnitude (g)'].max():.4f}g")
print(f"   • Average: {bottoms_df['Magnitude (g)'].mean():.4f}g")
print("   • Clear minima make rep counting reliable")
print("   • Some variation in depth between reps (normal)")

print("\n4️⃣  CONCENTRIC PHASE (Press)")
avg_power = peaks_df['Power above Baseline'].mean()
max_power = peaks_df['Power above Baseline'].max()
print(f"   • Average power: {avg_power:.4f}g above baseline")
print(f"   • Max power: {max_power:.4f}g above baseline")
print(f"   • Peak magnitude: up to {peaks_df['Magnitude (g)'].max():.4f}g")
print("   • Shows explosive power generation")

print("\n5️⃣  TEMPO AND TIMING")
if len(rep_durations) > 0:
    tempo_cv = (rep_durations.std() / rep_durations.mean()) * 100
    print(f"   • Average rep duration: {rep_durations.mean():.2f}s")
    print(f"   • Coefficient of variation: {tempo_cv:.1f}%")
    if tempo_cv < 20:
        print("   • Tempo is CONSISTENT - good control")
    elif tempo_cv < 40:
        print("   • Tempo is MODERATE - acceptable variation")
    else:
        print("   • Tempo is VARIABLE - consider more consistent pacing")

print("\n6️⃣  COMPARISON TO SQUAT")
print("   SIMILARITIES:")
print("   • Same phase pattern: Eccentric → Bottom → Concentric")
print("   • Clear magnitude drop during descent")
print("   • Return to stable baseline after rep")
print("   ")
print("   DIFFERENCES:")
print("   • Shorter ROM (range of motion)")
print("   • Horizontal vs vertical movement plane")
print("   • NO ground impact events")
print("   • Potentially faster tempo")

print("\n7️⃣  DETECTION RELIABILITY")
print(f"   • {len(bottoms_idx)} reps detected")
print(f"   • {len(peaks_idx)} concentric peaks detected")
print("   • High signal-to-noise ratio")
print("   • Minimal false positives expected")
print("   • Suitable for real-time feedback systems")

print("\n" + "="*80)
print("✅ Analysis demonstrates bench press is highly suitable for")
print("   accelerometer-based tracking and form analysis")
print("="*80)